In [163]:
import cv2 as cv
import torch
import numpy as np


In [164]:
print(cv.__version__)

4.5.5


In [165]:
model = torch.hub.load( \
                      'ultralytics/yolov5', \
                      'yolov5s', \
                      pretrained=True)


Using cache found in C:\Users\PCX/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-2-18 torch 1.10.2+cu113 CUDA:0 (GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [166]:
print(model.names)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [167]:
def plot_boxes(results, frame):

        labels, cord = results
        
        n = len(labels)
        
        
        x_shape, y_shape = frame.shape[1], frame.shape[0]
        for i in range(n):
            row = cord[i]
            
            
            # If score is less than 0.2 we avoid making a prediction.
            if row[4] < 0.4:
                continue

            
            
           
            x1 = int(row[0]*x_shape)
            y1 = int(row[1]*y_shape)
            x2 = int(row[2]*x_shape)
            y2 = int(row[3]*y_shape)
            bgr = (0, 255, 0) # color of the box
            classes = model.names # Get the name of label index
            class_labels = int(labels[i])
            
            #print(type(classes[class_labels]))
            
            
            label_font = cv.FONT_HERSHEY_SIMPLEX #Font for the label.
            cv.rectangle(frame, \
                        (x1, y1), (x2, y2), \
                        bgr, 2) #Plot the boxes
            cv.putText(frame,\
                        (f"{classes[class_labels]}"), \
                        (x1, y1), \
                        label_font, 0.9, bgr, 2) #Put a label over box.
            return frame

In [168]:
def score_frame(frame, model):
    if frame is None:
        pass
    else:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
        model.to(device)
    
        results = model(frame)
    
        labels = results.xyxyn[0][:, -1].cpu().numpy()

        
        cord = results.xyxyn[0][:, :-1].cpu().numpy()
        return labels, cord

In [169]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Camera cannot be opened !")
    exit()

while True:

    # Capture the video frame
    
   
    ret, frame = cap.read()
    

    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        



    results = score_frame(gray_frame,model)
    final_frame = plot_boxes(results,gray_frame)


    # the first 2 image returns as None. So i will bypass this error    
    if final_frame is None:
         pass
    else:
        cv.imshow("final",final_frame)

    # the first 2 image returns as None. So i will bypass this error
   
        
        
        
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice

        
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
# Destroy all the windows
cv.destroyAllWindows()